## Imports y config

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [6]:
# Configurar el driver de Selenium
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--headless')            # sin interfaz gráfica
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 0)

## **1. Web Scraping**

In [7]:
def generar_urls_completas():
    # Lista de ciudades y departamentos (ampliada)
    ciudades_departamentos = [
        # Principales
        ("bogota", "bogota-dc"),
        ("medellin", "antioquia"),
        ("cali", "valle-del-cauca"),
        ("barranquilla", "atlantico"),
        ("cartagena", "bolivar"),
        ("pereira", "risaralda"),
        ("bucaramanga", "santander"),
        ("cucuta", "norte-de-santander"),
        ("ibague", "tolima"),
        ("villavicencio", "meta"),
        ("manizales", "caldas"),
        ("pasto", "narino"),
        ("monteria", "cordoba"),
        ("santa-marta", "magdalena"),
        ("armenia", "quindio"),
        
        # Municipios estratégicos
        ("soacha", "cundinamarca"),
        ("facatativa", "cundinamarca"),
        ("girardot", "cundinamarca"),
        ("zipaquira", "cundinamarca"),
        ("envigado", "antioquia"),
        ("sabaneta", "antioquia"),
        ("bello", "antioquia"),
        ("itagui", "antioquia"),
        ("rionegro", "antioquia"),
        ("palmira", "valle-del-cauca"),
        ("yumbo", "valle-del-cauca"),
        ("tulua", "valle-del-cauca"),
        ("soledad", "atlantico"),
        ("dosquebradas", "risaralda"),
        ("jamundi", "valle-del-cauca"),
        ("girardot", "cundinamarca"),
        ("cajica", "cundinamarca"),
        ("chia", "cundinamarca"),
        ("la-ceja", "antioquia"),
        ("el-retiro", "antioquia"),
        
        # Turísticas
        ("san-andres", "san-andres"),
        ("leticia", "amazonas"),
        ("melgar", "tolima"),
        ("santa-fe-de-antioquia", "antioquia")
    ]

    # Tipos de propiedades (individuales y combinados)
    tipos_propiedades = [
        "casas",
        "apartamentos",
        "fincas",
        "apartaestudios",
        "cabanas",
        "casas-campestres",
        "casas-lotes",
        "casas-y-apartamentos",
        "casas-y-fincas",
        "apartamentos-y-apartaestudios",
        "casas-campestres-y-cabanas",
        "casas-y-apartamentos-y-fincas-y-apartaestudios-y-cabanas-y-casas-campestres-y-casas-lotes"
    ]

    # URLs base
    urls = [
        # "https://www.fincaraiz.com.co/venta",
        # "https://www.fincaraiz.com.co/inmobiliarias",
        # "https://www.fincaraiz.com.co/constructoras"
    ]

    # Generar URLs por tipo de propiedad y ubicación
    for ciudad, departamento in ciudades_departamentos:
        # URLs generales por ciudad
        urls.append(f"https://www.fincaraiz.com.co/venta/{ciudad}/{departamento}")
        urls.append(f"https://www.fincaraiz.com.co/proyectos-vivienda/{ciudad}/{departamento}")
        
        # URLs por tipo de propiedad específico
        for tipo in tipos_propiedades:
            urls.append(f"https://www.fincaraiz.com.co/venta/{tipo}/{ciudad}/{departamento}")
        
        # URLs especiales para proyectos de vivienda
        urls.append(f"https://www.fincaraiz.com.co/proyectos-vivienda-nueva/{ciudad}/{departamento}")

    # Eliminar duplicados y mantener orden
    return list(dict.fromkeys(urls))

def scrape_portal(url):
    def scrape_detail(detail_url):
        driver.get(detail_url)
        # 1) Esperar a que cargue el título del detalle
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.property-title')))
        time.sleep(1)  # opcional: asegurar que toda la sección renderice

        soup_det = BeautifulSoup(driver.page_source, 'html.parser')
        detail = {}

        # -- Información del proyecto (clave: valor) --
        for li in soup_det.select('div.project-info ul.ant-list-items li.ant-list-item'):
            cols = li.select('div.ant-col')
            if len(cols) >= 2:
                key = cols[0].get_text(strip=True)
                val = cols[1].get_text(strip=True)
                detail[key] = val

        # -- Descripción completa --
        desc = soup_det.select_one('div.property-description')
        detail['Descripción completa'] = desc.get_text(strip=True) if desc else None

        # -- Unidades / Tipos --
        units = []
        for unit_li in soup_det.select('div.project-units-section ul.ant-list-items li.proyect_units_list_item'):
            u = {}
            for ui in unit_li.select('div.unit_item'):
                label = ui.select_one('strong').get_text(strip=True)
                # el texto restante tras el <strong>
                value = ui.get_text(strip=True).replace(label, '').strip()
                u[label] = value
            units.append(u)
        detail['Unidades'] = units

        return detail

    driver.get(url)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.listingCard')))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    inmuebles = soup.select('div.listingCard')

    datos = []
    for itm in inmuebles:
        info = {}
        # -- Resumen en listado (igual que antes) --
        cover = itm.select_one('a.lc-cardCover')
        if cover:
            info['Título'] = cover.get('title', '').strip()
            href = cover.get('href', '').strip()
            info['URL detalle'] = f"https://www.fincaraiz.com.co{href}"
        img = itm.select_one('img.card-image-gallery--img')
        info['URL imagen'] = img['src'].strip() if img else None
        info['Etiquetas'] = [t.get_text(strip=True) for t in itm.select('span.property-tag')]
        precio = itm.select_one('span.price')
        info['Precio listado'] = precio.get_text(strip=True) if precio else None
        tip = itm.select_one('div.lc-typologyTag span')
        info['Tipología listado'] = tip.get_text(' ', strip=True) if tip else None
        desc = itm.select_one('span.lc-title')
        info['Descripción breve'] = desc.get_text(strip=True) if desc else None
        loc = itm.select_one('strong.lc-location')
        info['Ubicación listado'] = loc.get_text(strip=True) if loc else None
        pub = itm.select_one('div.publisher strong')
        info['Publicante'] = pub.get_text(strip=True) if pub else None
        btn = itm.select_one('div.property-lead-button button')
        info['Acción disponible'] = btn.get_text(strip=True) if btn else None

        # -- Scrape de la página de detalle y fusión --
        if info.get('URL detalle'):
            try:
                det = scrape_detail(info['URL detalle'])
                info.update(det)
            except Exception as e:
                info['Error detalle'] = str(e)

        datos.append(info)

    return datos


from typing import Callable, List, Dict, Sequence
import logging
import time

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)           # Cambia a DEBUG si lo necesitas
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s │ %(levelname)s │ %(message)s"))
logger.addHandler(handler)

def scrape_multiple_pages(
    base_url: str,
    *,
    scraper: Callable[[str], Sequence[Dict]],
    max_pages: int = 50,
    stop_on_empty: bool = True,
    stop_on_error: bool = True,
    delay: float | None = None,
) -> List[Dict]:
    """
    Recorre páginas secuenciales de un portal inmobiliario hasta que ocurre
    alguno de los eventos de parada configurables.

    Parameters
    ----------
    base_url : str
        URL de la primera página (sin el sufijo `/paginaX`).  
        E.g. ``"https://www.fincaraiz.com.co/venta/casas-lotes/bogota/bogota-dc"``.
    scraper : Callable[[str], Sequence[dict]]
        Función que recibe una URL y devuelve un iterable de dicts (un inmueble cada uno).
        Se inyecta para mantener la función pura y testeable.
    max_pages : int, default 50
        Máximo de páginas a visitar (incluyendo la primera).
    stop_on_empty : bool, default True
        Si es *True*, se detiene cuando `scraper` devuelve una colección vacía.
    stop_on_error : bool, default True
        Si es *True*, se detiene ante la primera excepción lanzada por `scraper`.
        Si es *False*, solo registra el error y continúa.
    delay : float or None, default None
        Segundos a esperar entre peticiones; útil para ser amable con el servidor
        y evitar *rate-limits*.

    Returns
    -------
    List[dict]
        Lista agregada con todos los inmuebles encontrados hasta la parada.
    """
    all_listings: List[Dict] = []

    for page_num in range(1, max_pages + 1):
        url = base_url if page_num == 1 else f"{base_url}/pagina{page_num}"
        logger.info("Scraping página %d: %s", page_num, url)

        try:
            page_data = list(scraper(url))
        except Exception as exc:             # noqa: BLE001  (cazar la excepción base es OK aquí)
            logger.error("Error en página %d → %s", page_num, exc)
            if stop_on_error:
                break
            continue

        if not page_data:
            logger.warning("Página %d sin resultados. Fin del scraping.", page_num)
            if stop_on_empty:
                break

        all_listings.extend(page_data)

        if delay:
            time.sleep(delay)

    logger.info("Scraping finalizado. %d inmuebles recopilados.", len(all_listings))
    return all_listings


In [25]:
url = "https://www.fincaraiz.com.co/venta/casas-lotes/bogota/bogota-dc"
data = scrape_multiple_pages(
            base_url=url ,
            scraper=scrape_portal,
            max_pages = 2)

2025-07-06 12:13:34,431 │ INFO │ Scraping página 1: https://www.fincaraiz.com.co/venta/casas-lotes/bogota/bogota-dc
2025-07-06 12:13:34,431 │ INFO │ Scraping página 1: https://www.fincaraiz.com.co/venta/casas-lotes/bogota/bogota-dc
2025-07-06 12:15:23,329 │ INFO │ Scraping página 2: https://www.fincaraiz.com.co/venta/casas-lotes/bogota/bogota-dc/pagina2
2025-07-06 12:15:23,329 │ INFO │ Scraping página 2: https://www.fincaraiz.com.co/venta/casas-lotes/bogota/bogota-dc/pagina2
2025-07-06 12:16:57,303 │ INFO │ Scraping finalizado. 42 inmuebles recopilados.
2025-07-06 12:16:57,303 │ INFO │ Scraping finalizado. 42 inmuebles recopilados.


In [31]:
# Ejemplo de uso
todas_urls = generar_urls_completas()
print(f"Total de URLs generadas: {len(todas_urls)}")

# Guardar en archivo
with open("urls_fincaraiz.txt", "w") as f:
    for url in todas_urls:
        f.write(url + "\n")

Total de URLs generadas: 570


In [32]:
todas_urls = todas_urls[:3]  # Limitar a las primeras 10 para pruebas
todas_urls

['https://www.fincaraiz.com.co/venta/bogota/bogota-dc',
 'https://www.fincaraiz.com.co/proyectos-vivienda/bogota/bogota-dc',
 'https://www.fincaraiz.com.co/venta/casas/bogota/bogota-dc']

In [33]:
for url in todas_urls:
    print(f"Scraping URL: {url}")
    try:
        data = scrape_multiple_pages(
            base_url=url,
            scraper=scrape_portal,      # tu función que extrae datos de una página
            max_pages=50,               # límite superior (por defecto es 50)
            stop_on_empty=True,         # se detiene si una página no tiene resultados
            stop_on_error=True,         # se detiene si ocurre una excepción
            delay=0.0                   # espera 2 segundos entre páginas para no saturar el servidor
        )
        df = pd.DataFrame(data)
        
        # Extraer los segmentos de la URL desde la posición -4 hasta -1
        partes_url = url.split('/')
        if len(partes_url) < 4:
            raise ValueError("La URL no tiene suficientes segmentos para generar el nombre del archivo.")
        
        nombre_archivo = f"datos_{'_'.join(partes_url[-4:-1])}.csv"
        
        # Guardar los datos en un archivo CSV
        df.to_csv(nombre_archivo, index=False, encoding='utf-8')
        print(f"Datos guardados en {nombre_archivo}")
        
    except Exception as e:
        print(f"Error al procesar {url}: {e}")


2025-07-06 12:24:29,263 │ INFO │ Scraping página 1: https://www.fincaraiz.com.co/venta/bogota/bogota-dc
2025-07-06 12:24:29,263 │ INFO │ Scraping página 1: https://www.fincaraiz.com.co/venta/bogota/bogota-dc


Scraping URL: https://www.fincaraiz.com.co/venta/bogota/bogota-dc


2025-07-06 12:25:48,869 │ INFO │ Scraping página 2: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina2
2025-07-06 12:25:48,869 │ INFO │ Scraping página 2: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina2
2025-07-06 12:27:08,675 │ INFO │ Scraping página 3: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina3
2025-07-06 12:27:08,675 │ INFO │ Scraping página 3: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina3
2025-07-06 12:28:33,914 │ INFO │ Scraping página 4: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina4
2025-07-06 12:28:33,914 │ INFO │ Scraping página 4: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina4
2025-07-06 12:30:24,601 │ INFO │ Scraping página 5: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina5
2025-07-06 12:30:24,601 │ INFO │ Scraping página 5: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/pagina5
2025-07-06 12:32:04,353 │ INFO │ Scraping página 6: https://www.fincaraiz.com.co/venta/bogota/bogota-dc/

KeyboardInterrupt: 

In [ ]:
base_url = 'https://www.fincaraiz.com.co/venta/ibague/tolima'
todas_las_propiedades = scrape_multiple_pages(base_url, max_pages=50)
df = pd.DataFrame(todas_las_propiedades)


In [9]:
import math

def tile_to_latlon(x, y, z):
    """
    Convierte coordenadas de tile XYZ a latitud y longitud.
    """
    n = 2 ** z
    lon_deg = x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

# Usar los valores de tu HTML
lat, lon = tile_to_latlon(19023, 31660, 16)
print(f"Ubicación aproximada: {lat:.6f}, {lon:.6f}")

                                                                Título                                                                                                              URL detalle                                                                                                                                                                                                                                  URL imagen                                Etiquetas       Precio listado           Tipología listado                                                 Descripción breve      Ubicación listado                                                         Publicante Acción disponible           Estado Estrato  Parqueaderos               Financiación Formas de pago Cuota inicial Pisos interiores Aplica subsidio                                                                                                                                                                                     